<a href="https://colab.research.google.com/github/Maya7991/gsc_classification/blob/main/snn_conv1d_mfcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SNN conv1D on MFCC

### Imports

In [68]:
!pip install snntorch --quiet
!pip install torchaudio --quiet

In [69]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchaudio
from torchaudio.datasets import SPEECHCOMMANDS
from torch.utils.data import DataLoader, Dataset
import torchaudio.transforms as T

from snntorch import spikegen, surrogate, functional as SF
import snntorch as snn

### Load & Preprocess the Speech Command Dataset

In [70]:
transform = torchaudio.transforms.MFCC(
    sample_rate=16000,
    n_mfcc=20,
    melkwargs={'n_fft': 400, 'hop_length': 160, 'n_mels': 40}
)

train_dataset = SPEECHCOMMANDS(
    "./", download=True, subset="training")
test_dataset = SPEECHCOMMANDS(
    "./", download=True, subset="testing")

# Limit to a few keywords for now (e.g., "yes", "no", "up", "down")
keywords = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']
label_dict = {k: i for i, k in enumerate(keywords)}

def collate_fn(batch):
    X, y = [], []
    max_len = 0
    mfccs = []

    for waveform, sample_rate, label, *_ in batch:
        if label in keywords:
            mfcc = transform(waveform).squeeze(0)  # [n_mfcc, time]
            mfccs.append(mfcc)
            # print("MFCC shape:", mfcc.shape)  # add this
            y.append(label_dict[label])
            max_len = max(max_len, mfcc.shape[1])

    for mfcc in mfccs:
        pad_len = max_len - mfcc.shape[1]
        padded = F.pad(mfcc, (0, pad_len))  # Pad on the time dimension (right side)
        X.append(padded)

    return torch.stack(X), torch.tensor(y)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)


### Latency encoding

encode mfcc features to spike trains

In [71]:
# def encode_input(mfcc_batch, num_steps=100):
#     # Normalize to [0, 1]
#     data = (mfcc_batch - mfcc_batch.min()) / (mfcc_batch.max() - mfcc_batch.min())
#     # [B x C x L] → [B x L] if needed
#     data = data.mean(dim=1) if data.ndim == 3 else data
#     # Apply latency encoding
#     spk_data = spikegen.latency(data, num_steps=num_steps, normalize=True, linear=True)
#     return spk_data  # shape: [T x B x L]

def encode_input(mfcc_batch, num_steps=100):
    # Normalize to [0, 1] per sample
    min_val = mfcc_batch.amin(dim=(1,2), keepdim=True)
    max_val = mfcc_batch.amax(dim=(1,2), keepdim=True)
    data = (mfcc_batch - min_val) / (max_val - min_val + 1e-7)

    # Shape: [B, C, L] → [T, B, C, L]
    spk_data = spikegen.latency(data, num_steps=num_steps, normalize=True, linear=True)
    return spk_data  # [T, B, C, L]

### Conv1D SNN Architecture

In [72]:
class SNNConv1D(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        beta = 0.9  # LIF decay constant
        self.conv1 = nn.Conv1d(20, 32, kernel_size=5, stride=1, padding=2)
        self.lif1 = snn.Leaky(beta=beta, spike_grad=surrogate.fast_sigmoid())

        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.lif2 = snn.Leaky(beta=beta, spike_grad=surrogate.fast_sigmoid())

        # self.fc1 = nn.Linear(64 * 20, num_classes)
        self.fc1 = nn.Linear(64 * 101, num_classes)
        self.num_steps = 100

    def forward(self, x):
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        spk_out = 0

        for step in range(self.num_steps):
            input_t = x[step]  # Shape: [B x input_dim] [B, 20, T]
            # input_t = input_t.unsqueeze(1)  # Add channel dim → [B x 1 x L]
            x1 = self.conv1(input_t)
            spk1, _ = self.lif1(x1)

            x2 = self.conv2(spk1)
            spk2, _ = self.lif2(x2)

            x_flat = spk2.view(spk2.size(0), -1)
            # print("Flattened shape:", x_flat.shape)
            out = self.fc1(x_flat)
            spk_out += out
        return spk_out / self.num_steps  # Soft output across time


### Training Loop

In [73]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SNNConv1D(num_classes=len(keywords)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

def train_epoch(model, loader):
    model.train()
    total_loss = 0
    correct = 0
    for x, y in loader:
        # x = transform(x).to(device)  # Apply MFCC transform
        spk_x = encode_input(x, num_steps=model.num_steps).to(device)  # [T x B x L] -> TTFS encoding
        y = y.to(device)

        optimizer.zero_grad()
        out = model(spk_x)
        loss = loss_fn(out, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (out.argmax(dim=1) == y).sum().item()
    acc = correct / len(loader.dataset)
    return total_loss / len(loader), acc


### Train & Evaluate

In [ ]:
for epoch in range(10):
    train_loss, train_acc = train_epoch(model, train_loader)
    print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Accuracy={train_acc*100:.2f}%")
    if train_acc > 0.90:
        print("Target accuracy reached!")
        break
